In [ ]:
import sims
import numpy as np
import msprime
import pandas as pd

import useful
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt
from importlib import reload


import json

In [ ]:
# Generation time, mutation rate and recomination rate
RR = 1e-8
MU = 1.29e-8 
GEN_time = 29.0 

# Split Times
T_NEAND_migration = 55000 #time of Neanderthal migration into Out_of_africa population
T_NEAND_AMH = 550000 # split time between AMH and Neanderthal
T_OOF_AF = 65700 # Out_of_Africa migration time
T_NEAND_samples = 38000

# Effective population size
N_ANC = 18500 # N_e of common  AMH and NEanderthal population 
N_ND = 3400 # N_e of Neanderthal
N_AMH = 23000 # N_e of AMH
N_OOF = 1861 # N_e of Out_of_Africa population
N_AF = 27600 # N_e of Africans
N_EU = 13377 #N_e of Europeans

N_EU_bottleneck = 1080
N_EU_growth = 1450
T_EU_growth = 31900
gr_rate = 0.00202
Portion_admix = 0.03

len_sequence = 50000000 # DNA sequence length

n = 250 # number of generated   AF samples
n_neand = 6 #number of generated Neanderthals

rand_sd =1234 #random seed
T = np.array([T_NEAND_migration, T_NEAND_AMH, T_OOF_AF])/GEN_time


N_ND = 3400 # N_e of Neanderthal
N_e = np.array([N_ANC, N_ND, N_AMH, N_OOF, N_AF, N_EU])

n_eu=1
ploidy=2
L=1000
N_ref_pop=250
N_neanderthal=6

In [ ]:
#functions that move the j-th set of tracts by j*len_sequence
def many_in_one(nd_tracts_mas, number_chr, length):
    nd2=[]
    for _ in range(len(nd_tracts_mas)):
        for j in range(len(nd_tracts_mas[_])):
            nd2.append([nd_tracts_mas[_][j][0]+_*length, nd_tracts_mas[_][j][1]+_*length])

    return [useful.tracts_eu(nd2, length*number_chr ), nd2]

# Create dataframe

In [ ]:
number_chr=60
length=50000000

dir='./sims3/'
#read real
r=[]
real_tracts=[]
for j in range(1, number_chr+1):
    a=str(np.load(dir+str(j)+'.ND_true_tracts.npy', allow_pickle= True))

    if 'list' in a:

    
        b=str(a).strip().replace('list','')[4:-4].replace('])','').replace(' ([' ,'').split('\n')
        for jj in b[0].split('], ['):      
            real_tracts.append([int(float(_))+(j-1)*length for _ in jj.replace('[','').replace(']','').split(', ')])
            

    else:
        for jj in a[4:-4].split('\n\n')[0].replace(']','').replace('[','').split('\n   '):
            real_tracts.append([float(jj.strip().split(' ')[0])+(j-1)*length, float(jj.split(' ')[1])+(j-1)*length])
real_tracts_in_states = [useful.exclude_gaps([[0, number_chr*length]], real_tracts),real_tracts]


#read daiseg
tracts_daiseg_mas=[]
df_daiseg_mas=[]
cover_mas=[0.25, 0.4, 0.5, 0.65, 0.8, 0.9, 0.999]
#cover_mas=[0.25,  0.5,  0.8,  0.999]
for cover in cover_mas:
    daiseg=[]
    for _ in range(number_chr):
        daiseg.append(useful.read_out(dir+'out.chr'+str(_+1)+'.cover.'+str(cover)+'.archaic.txt')[0])
    tracts_daiseg = many_in_one(daiseg, number_chr, length)
    tracts_daiseg_mas.append(tracts_daiseg)

    df_daiseg_mas.append(sims.df_result_lonf_chr(real_tracts_in_states, tracts_daiseg, N_neanderthal,  N_ref_pop, 2))



# read hmmix_results
df_hmmix, hmmix_mas=[],[]
#read hmmix tracts
cut_mas=[0.85, 0.9, 0.95]

for cut in cut_mas:

    hmmmix_tr=[[] for j in range(number_chr)]
    for j in range(1, number_chr+1):
        file=dir+'out.chr'+str(j)+'.hap1.txt'    
        hmmmix_tr[j-1]=sims.read_noND2(file, cut )


        
    hmmix=many_in_one(hmmmix_tr,number_chr, length)
    hmmix_mas.append(hmmix)
    df_hmmix.append(sims.df_result_lonf_chr(real_tracts_in_states, hmmix, N_neanderthal,  N_ref_pop, 2))


#create dataframe with results
bounds=[[0, 10],[10,20],[20,40],[40, 60], [60, 100], [100, 5000]]
labels=[i for i in range(len(bounds))]
df=pd.DataFrame(columns=['CHR', 'POS','Marker', 'Length', 'Int_with_real', 'Int_count', '%Truth', 'Length_category'])
tracts=[real_tracts_in_states[1]]+[hmmix_mas[_][1] for _ in range(len(cut_mas))]+[tracts_daiseg_mas[_][1] for _ in range(len(cover_mas))]
markers=['real']+['hmmix_'+str(cut) for cut in cut_mas] + ['daiseg_'+str(cover) for cover in cover_mas]

for _, m in zip(tracts, markers):  
    for j in _:
        #counting number of intersections        
        k=0
        for i in tracts[0]:
            if useful.intersections([j], [i]):
                k+=1
        for b in bounds:            
            if (j[1]-j[0]+1)/1000>=b[0] and (j[1]-j[0]+1)/1000<=b[1]:
                length_cat=labels[bounds.index(b)]                
        if k==0:
            df.loc[len(df)]=[int(j[1]//length)+1,j,m, (j[1]-j[0]+1)/1000, 0, k, 0, length_cat]
        else:            
            df.loc[len(df)]=[int(j[1]//length)+1,j,m, (j[1]-j[0]+1)/1000, 1, k, 
                             useful.len_tracts(useful.intersections([j], tracts[0])), length_cat]


In [ ]:
#proportion of false-positive length    in all length and in class 

for cut in cut_mas:

    print('The dosage of false positive length in all tract length ', 'hmmix_'+str(cut),
          np.array(df[(df['Marker']=='hmmix_'+str(cut)) & (df['Int_count']==0)]['Length']).sum()/np.array(df[(df['Marker']=='hmmix_'+str(cut))]['Length']).sum())



for cover in cover_mas:

    print('The dosage of false positive length in all tract length ', 'daiseg_'+str(cover), 
          np.array(df[(df['Marker']=='daiseg_'+str(cover)) & (df['Int_count']==0)]['Length']).sum()/np.array(df[(df['Marker']=='daiseg_'+str(cover))]['Length']).sum())

# Hist plots

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 9), sharey=True, sharex='all')
#fig.suptitle('Length distribution of tracts')
sns.histplot(ax=axes[0][0], data=df[df['Marker']=='real'], x='Length', alpha=0.6, binwidth=10 ,  kde=True, color='orange')
sns.histplot(ax=axes[0][1], data=df[df['Marker']=='hmmix_0.85'], x='Length', alpha=0.6, binwidth=10,  kde=True, hue='Int_with_real',multiple='stack', hue_order=[0,1]  )
sns.histplot(ax=axes[0][2], data=df[df['Marker']=='hmmix_0.95'], x='Length', alpha=0.6, binwidth=10,  kde=True, hue='Int_with_real',multiple='stack')
sns.histplot(ax=axes[1][0], data=df[df['Marker']=='daiseg_0.25'], x='Length', alpha=0.6, binwidth=10,  kde=True, hue='Int_with_real',multiple='stack')
sns.histplot(ax=axes[1][1], data=df[df['Marker']=='daiseg_0.65'], x='Length', alpha=0.6, binwidth=10,  kde=True, hue='Int_with_real',multiple='stack')

sns.histplot(ax=axes[1][2], data=df[df['Marker']=='daiseg_0.999'], x='Length', alpha=0.6, binwidth=10,  kde=True, hue='Int_with_real',multiple='stack')


axes[0][0].set_title('Real')
axes[0][1].set_title('HMMmix_0.85')
axes[0][2].set_title('HMMmix_0.95')
axes[1][0].set_title('DAIseg_0.25')

axes[1][1].set_title('DAIseg_0.65')

axes[1][2].set_title('DAIseg_1.0')
plt.savefig('Length.distribution.eps', format='eps', transparent=True)
plt.show()



# Deviation from mean tract length by chr

In [ ]:
#deviation of mean by chr

x=[_ for _ in range(1,number_chr+1)]
means={}
for m in list(set(df['Marker'])):
    d=df[df['Marker']==m][['CHR','Length']]    
    means[m]=np.array(d.groupby(['CHR']).mean()['Length'])

fig, ax = plt.subplots( figsize=(5, 5))

ax.plot(x, sorted([(x-y)/y for x,y in zip(means['daiseg_0.999'],means['real'])]), label='daiseg_1.0')
ax.plot(x, sorted([(x-y)/y for x,y in zip(means['hmmix_0.9'],means['real'])]), label='hmmix_0.9')
ax.plot(x, sorted([(x-y)/y for x,y in zip(means['daiseg_0.5'],means['real'])]), label='daiseg_0.5')
ax.plot(x, sorted([(x-y)/y for x,y in zip(means['daiseg_0.8'],means['real'])]), label='daiseg_0.8')
#plt.plot(x, [(x-y)/y for x,y in zip(df_means['daiseg'],df_means['real'])], label='daiseg')


ax.set_ylabel('Deviation from the real mean length divided by the real mean')
ax.set_xlabel(' CHR')
plt.legend()

#plt.savefig('deviation.of.mean.eps', format='eps', transparent=True)

#plt.show()


# Precision/recall by length

In [ ]:
cover_labels=['hmmix_'+str(0.9)]+['daiseg_'+str(_) for _ in cover_mas]

precision=[]
for _ in cover_labels:

    df2=df[df['Marker']==_]   
    df2.head()
    s=[0 for j in range(len(bounds))]
    for b in range(len(labels)):        
        s[b]=(df2[df2['Length_category']==labels[b]]['%Truth'].sum())/(df2[df2['Length_category']==labels[b]]['Length'].sum()*1000)

    precision.append(s)

precision=np.array(precision)
precision= precision.transpose() 

recall=[]
df1=df[df['Marker']=='real']
df2=[df[df['Marker']==j] for j in cover_labels]
for l in labels:
    length_cover=[0 for _ in cover_labels]
    length_full=0
    lc_m2=[]
    for j in df1[df1['Length_category']==l]['POS']:     
        length_full += j[1]- j[0]
        for c in range(len(cover_labels) ):
            df22=df2[c]        
            for i in df22['POS']:
                length_cover[c]+=useful.len_tracts(useful.intersections([j], [i]))

    recall.append([length_cover[__]/length_full for __ in range(len(cover_labels))])
    
recall=np.array(recall)
            
                



fig, axes = plt.subplots(1, 2, figsize=(12, 5), sharey=True, sharex=True)
for s, k in zip(['pr'], [1]):
    for j, c in zip(range(len(bounds)), ['r', 'g', 'b', 'c', 'm', 'y', 'salmon']):        
        axes[0].plot([0]+[_ for _ in cover_mas], precision[j],    markersize=1, label=str(bounds[j]))
        axes[1].plot([0]+[_ for _ in cover_mas], recall[j],    markersize=1, label=str(bounds[j]))

axes[0].set_xlabel("Portion of Neanderthal coverage")
axes[1].set_xlabel("Portion of Neanderthal coverage")
axes[0].set_ylabel('% of the truth in hmm ND tracts')
axes[1].set_ylabel('% hmm ND tracts in real tracts')



l, lines=[],[]
for ax in fig.axes: 
    Line, Label = ax.get_legend_handles_labels() #

    lines.extend(Line) 
    l.extend(Label) 
    
    break    
fig.legend(lines, l, loc='upper right') 
plt.savefig('Length.quality.eps', format='eps', transparent=True)

plt.show()

# Merged pairs

In [ ]:
df2=df[(df['Marker']=='real')]

dist=40000
pair_set=[] # set of neghboring real tracts
for j in range(len(df2['POS'])-1):
    if -df2['POS'][j][1]+df2['POS'][j+1][0] < dist :
        pair_set.append([df2['POS'][j],df2['POS'][j+1] ])

for l in ['hmmix_'+str(cut) for cut in cut_mas] + ['daiseg_'+str(cover) for cover in cover_mas]:
    c=0
    for ps in pair_set:
        for j in df[df['Marker']==l]['POS']:            
            if useful.intersections([ps[0]],[j])!=[] and useful.intersections([ps[1]],[j])!=[]:
                c+=1

    print('% of neighboring pairs that gluing together',l,round(c/len(pair_set),3))
    
  